In [ ]:
import os, requests, zipfile, re
from collections import Counter

import numpy as np

In [ ]:
dataset_url = "https://github.com/entropicemergence/tiny_llm_server/releases/download/v0.1.0/TinyStoriesV2-GPT4-small.zip"
datset_folder = "stories"
os.makedirs(datset_folder, exist_ok=True)

def download_dataset(url):
    filename = url.split("/")[-1]
    filepath = os.path.join(datset_folder, filename)
    if not os.path.exists(filepath):
        response = requests.get(url)
        with open(filepath, "wb") as f:
            f.write(response.content)
download_dataset(dataset_url)

In [ ]:
filepath = "lec2/stories/TinyStoriesV2-GPT4-small.zip"
with zipfile.ZipFile(filepath, "r") as zip_ref:
    zip_ref.extractall(datset_folder)
with open("lec2/stories/TinyStoriesV2-GPT4-small.txt", "r", encoding="utf-8") as f:
    stories_text = f.read()

In [ ]:
# print (stories_text[:10000])
text_split = stories_text.split("<|endoftext|>")

simbol = r'[.,!?;:"\'\-\(\)\[\]{}]'
replacement = r' \g<0> '


print ("total paragraf",len(text_split))
total_word_list = []
char_list = set()


for split in text_split:
    split = split.lower()
    split = re.sub(simbol, replacement, split)
    char_list.update(split)
    word_list = [word for word in split.split()]
    total_word_list.extend(word_list)
    # # print(split)
    # print (word_list)
    # print("-"*100)

# a = 23
# in = 50
# have =100

# print (total_word_list)
word_count = Counter(total_word_list)
print (len(word_count))
print (word_count)

top_word_count = word_count.most_common(1000)
print (len(top_word_count))
print (top_word_count)

print(len(char_list))
print(char_list)



total paragraf 27631
10203
Counter({'.': 445081, 'the': 271187, ',': 241407, 'and': 200521, 'a': 154699, 'to': 150710, '"': 119640, 'was': 108022, 'they': 86379, 'he': 83347, 'it': 70765, 'she': 60665, 'said': 43901, 'with': 43315, 'day': 43244, 'tim': 42915, 'his': 42650, 'her': 42103, "'": 40743, 'in': 40742, '!': 38617, 'you': 36537, 'big': 35140, 'but': 30046, 'one': 29058, 'had': 28998, 'that': 27345, 'not': 27110, 'i': 26953, 'mom': 26483, 'on': 26407, 'happy': 25946, 'of': 25867, 'very': 25476, 'saw': 24934, 'lily': 24847, 's': 24692, 'play': 24438, 'little': 24180, 'so': 23291, 'there': 22438, 'tom': 22423, 'for': 22023, 'time': 21995, 'named': 21072, 'were': 20113, 'wanted': 19991, 'friends': 19501, 'all': 18101, 'went': 17676, 'once': 17672, 'bird': 17334, 'is': 16666, 'upon': 16637, 'can': 15802, 'together': 14871, 'at': 14414, 'fun': 14002, '?': 13877, 'help': 13435, 'be': 13171, 'dog': 13125, 'their': 13022, 'cat': 12812, 'girl': 12415, 'played': 12386, 'are': 11887, 'up':

In [ ]:
kamus_kata = {}
kamus_char = {}

start_id = 0
for word, count in top_word_count:
    kamus_kata[word] = start_id
    start_id += 1

for char in char_list:
    kamus_char[char] = start_id
    start_id += 1

print (len(kamus_kata))
print (kamus_kata)

print (len(kamus_char))
print (kamus_char)

1000
{'.': 0, 'the': 1, ',': 2, 'and': 3, 'a': 4, 'to': 5, '"': 6, 'was': 7, 'they': 8, 'he': 9, 'it': 10, 'she': 11, 'said': 12, 'with': 13, 'day': 14, 'tim': 15, 'his': 16, 'her': 17, "'": 18, 'in': 19, '!': 20, 'you': 21, 'big': 22, 'but': 23, 'one': 24, 'had': 25, 'that': 26, 'not': 27, 'i': 28, 'mom': 29, 'on': 30, 'happy': 31, 'of': 32, 'very': 33, 'saw': 34, 'lily': 35, 's': 36, 'play': 37, 'little': 38, 'so': 39, 'there': 40, 'tom': 41, 'for': 42, 'time': 43, 'named': 44, 'were': 45, 'wanted': 46, 'friends': 47, 'all': 48, 'went': 49, 'once': 50, 'bird': 51, 'is': 52, 'upon': 53, 'can': 54, 'together': 55, 'at': 56, 'fun': 57, '?': 58, 'help': 59, 'be': 60, 'dog': 61, 'their': 62, 'cat': 63, 'girl': 64, 'played': 65, 'are': 66, 'up': 67, 'did': 68, 'him': 69, 't': 70, 'ball': 71, 'sue': 72, 'too': 73, 'could': 74, 'them': 75, 'sad': 76, 'boy': 77, 'tree': 78, 'max': 79, 'have': 80, 'loved': 81, 'found': 82, 'looked': 83, 'from': 84, 'then': 85, 'when': 86, 'friend': 87, 'go': 8

In [ ]:
#token  = kata atau character dalam tokenizer

class Tokenizer:
    def __init__(self, text_source):
        self.text_source = text_source
        self.max_word_vocab = 1000
        self.special_tokens = {'<UNKNOWN>':int(0), '<BOS>':int(1), '<EOS>':int(2), '<CHAR_START>':int(3), '<CHAR_END>':int(4)}
        self.punctuation = r'[.,!?;:"\'\-\(\)\[\]{}]'
        self.replacement = r' \g<0> '
        self.word_to_id = {}
        self.id_to_word = {}
        self.char_to_id = {}
        self.id_to_char = {}

        self.vocab_size =0

    def build_vocab(self):
        text_split = self.text_source.split("<|endoftext|>")

        total_word_list = []
        char_list = set()
        for split in text_split:
            split = split.lower()
            split = re.sub(self.punctuation, self.replacement, split)
            char_list.update(split)
            word_list = [word for word in split.split()]
            total_word_list.extend(word_list)
        
        word_count = Counter(total_word_list)
        start_id = 5
        top_word_count = word_count.most_common(self.max_word_vocab)
        for word, count in top_word_count:
            self.word_to_id[word] = start_id
            self.id_to_word[start_id] = word
            start_id += 1
        
        for char in char_list:
            self.char_to_id[char] = start_id
            self.id_to_char[start_id] = char
            start_id += 1
        
        self.vocab_size = start_id
    
    def encode(self, text):
        text = text.lower()
        text = re.sub(self.punctuation, self.replacement, text)
        word_list = [word for word in text.split()]
        token_list =[]
        token_list.append(self.special_tokens['<BOS>'])
        for word in word_list:
            if word in self.word_to_id:
                token_list.append(self.word_to_id[word])
            else:
                token_list.append(self.special_tokens['<CHAR_START>'])
                for char in word:
                    if char in self.char_to_id:
                        token_list.append(self.char_to_id[char])
                    else:
                        token_list.append(self.special_tokens['<UNKNOWN>'])
                token_list.append(self.special_tokens['<CHAR_END>'])
        return token_list
    
    def decode(self, token_list):
        text = ""
        for token in token_list:
            if token > 4:
                if token in self.id_to_word:
                    text += self.id_to_word[token] + " "
                else:
                    text += self.id_to_char[token]
            if token == self.special_tokens['<CHAR_END>']:
                text += " "
        return text

tokenizer = Tokenizer(stories_text)
tokenizer.build_vocab()

print (tokenizer.vocab_size)
print (tokenizer.special_tokens)
print (tokenizer.word_to_id)
print (tokenizer.char_to_id)


encoded_token = tokenizer.encode("ani is walking to the store")
print ("encoded token",encoded_token)

decoded_text = tokenizer.decode(encoded_token)
print ("decoded text",decoded_text)


# print (tokenizer.id_to_char[1063])
# print (tokenizer.id_to_char[1029])
# print (tokenizer.id_to_char[1012])

# print (tokenizer.id_to_word[57])
# print (tokenizer.id_to_word[430])
# print (tokenizer.id_to_word[10])
# print (tokenizer.id_to_word[6])
# print (tokenizer.id_to_word[349])

1067
{'<UNKNOWN>': 0, '<BOS>': 1, '<EOS>': 2, '<CHAR_START>': 3, '<CHAR_END>': 4}
{'.': 5, 'the': 6, ',': 7, 'and': 8, 'a': 9, 'to': 10, '"': 11, 'was': 12, 'they': 13, 'he': 14, 'it': 15, 'she': 16, 'said': 17, 'with': 18, 'day': 19, 'tim': 20, 'his': 21, 'her': 22, "'": 23, 'in': 24, '!': 25, 'you': 26, 'big': 27, 'but': 28, 'one': 29, 'had': 30, 'that': 31, 'not': 32, 'i': 33, 'mom': 34, 'on': 35, 'happy': 36, 'of': 37, 'very': 38, 'saw': 39, 'lily': 40, 's': 41, 'play': 42, 'little': 43, 'so': 44, 'there': 45, 'tom': 46, 'for': 47, 'time': 48, 'named': 49, 'were': 50, 'wanted': 51, 'friends': 52, 'all': 53, 'went': 54, 'once': 55, 'bird': 56, 'is': 57, 'upon': 58, 'can': 59, 'together': 60, 'at': 61, 'fun': 62, '?': 63, 'help': 64, 'be': 65, 'dog': 66, 'their': 67, 'cat': 68, 'girl': 69, 'played': 70, 'are': 71, 'up': 72, 'did': 73, 'him': 74, 't': 75, 'ball': 76, 'sue': 77, 'too': 78, 'could': 79, 'them': 80, 'sad': 81, 'boy': 82, 'tree': 83, 'max': 84, 'have': 85, 'loved': 86, 'f

In [ ]:
a1= np.array([1,2,3])
a2= np.array([1.1,2.1,3.1])


c1= np.array([-1,-1.5,0.2])

cosine_similarity_a = np.dot(a1,a2) / (np.linalg.norm(a) * np.linalg.norm(b))
print(cosine_similarity_a)

cosine_similarity_a_c1 = np.dot(a1,c1) / (np.linalg.norm(a1) * np.linalg.norm(c1))
print(cosine_similarity_a_c1)